In [1]:
import numpy as np
import tensorflow as tf
import cv2
from glob import glob

In [2]:
train_path = 'data/train'
test_path = 'data/test'
valid_path = 'data/valid'

In [3]:
from sklearn.datasets import load_files

In [4]:
def extraction(file_path):
    f = load_files(file_path)
    return f['filenames'],f['target']

In [5]:
train_files,train_targets = extraction(train_path)
test_files,test_targets = extraction(test_path)
valid_files,valid_targets = extraction(valid_path)

In [6]:
# in this cell i will onehot encode the targets.
from keras.utils import to_categorical
def onehot(targets):
    data = targets
    encoded = to_categorical(data)
    return encoded

Using TensorFlow backend.


In [7]:
# onehot encoded the targets
train_targets = onehot(train_targets)
valid_targets = onehot(valid_targets)
test_targets = onehot(test_targets)

In [136]:
# now in this cell and following cells i will convert the images to tensors
from keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
import cv2
def split_3_channels(image):
    image = load_img(image)
    img = img_to_array(image)
    img = cv2.resize(img,(512,512))/255.
    return img
def convert_tensor(images):
    tensors = list()
    for i in images:
        img = split_3_channels(i)
        tensors.append(img)
    tensors = np.array(tensors)
    return tensors
    

In [137]:
train_tensors = convert_tensor(train_files)
test_tensors = convert_tensor(test_files)
valid_tensors = convert_tensor(valid_files)

In [138]:
print(len(train_tensors))
print(test_tensors.shape)
print(valid_tensors.shape)

2000
(600, 512, 512, 3)
(150, 512, 512, 3)


In [ ]:
# in this this and following cells i will be doing image augumentation
from keras.preprocessing.image import ImageDataGenerator
datagen_train = ImageDataGenerator(rotation_range=120,width_shift_range = 0.2,height_shift_range=0.2,shear_range=0.2,zoom_range=0.5,horizontal_flip=True)
datagen_train.fit(train_tensors)

In [139]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
% matplotlib inline
import numpy as np

In [ ]:
fig = plt.figure(figsize=(30,5))
for i in range(12):
    ax = fig.add_subplot(1,12,i+1,xticks=[],yticks=[])
    ax.imshow(np.squeeze(train_tensors[i]))

In [ ]:
min_batch_x = train_tensors[:12]
min_batch_y = train_targets[:12]

fig = plt.figure(figsize=(30,5))
for x_batch in datagen_train.flow(min_batch_x,batch_size=12,save_to_dir='preview',save_format='jpeg'):
    for i in range(0,12):
        ax = fig.add_subplot(1,12,i+1,xticks=[],yticks=[])
        ax.imshow(np.squeeze(x_batch[i]))
    break;

## CNN in tensorflow


In [140]:
import tensorflow as tf

In [141]:
learning_rate = 0.0001
epochs = 3
batch_size = 250
n_classes = 3
dropout = 0.75

In [165]:
def next_batch1(inp,batch,batch_size):
    start = batch*batch_size
    end = start+batch_size
    return inp[start:end]
def create_biases(layer):
    return tf.Variable(tf.random_uniform([layer.get_shape().as_list()[-1]],minval=0.25,maxval=0.75))
# creating weights for dense layer
def create_dense(conv_layer,fc_depth):
    return tf.Variable(tf.random_uniform([conv_layer.get_shape().as_list()[1]*conv_layer.get_shape().as_list()[2]*conv_layer.get_shape().as_list()[3],fc_depth],minval=0.25,maxval=0.75))
def create_filter(kernel_size,input_depth,output_depth):
    return tf.Variable(tf.random_uniform([kernel_size,kernel_size,input_depth,output_depth],minval=0.25,maxval=0.75))
def conv_2d(x,input_depth,output_depth):
    #filter = create_filter(kernel_size,input_depth,output_depth)
    kernelsize = 3
    layer = tf.nn.conv2d(x,filter=create_filter(kernelsize,3,output_depth),strides=(1,1,1,1),padding='SAME')
    layer = tf.nn.bias_add(layer,create_biases(layer).get_shape().as_list()[-1])
    return layer
def max_pool(value):
    ksize = [1,2,2,1]
    strides = [1,2,2,1]
    padding = 'VALID'
    layer = tf.nn.max_pool(value,ksize=ksize,strides=strides,padding=padding)
    return layer
def fc_to_out_weights(fc_layer,out_depth):
    return tf.Variable(tf.random_uniform([fc_layer.shape[-1].value,out_depth],minval=0.25,maxval=0.75))

In [166]:
def cov_Net(image_batch,dropout):
    kernel_size = 3
    conv1 = conv_2d(image_batch,3,16)
    max1 = max_pool(conv1)
    conv2 = conv_2d(max1,16,32)
    max2 = max_pool(conv2)
    conv3 = conv_2d(max3,32,64)
    max3 = max_pool(conv3)
    conv4 = conv_2d(max3,64,128)
    max4 = max_pool(conv4)
    conv5 = conv_2d(max4,128,256)
    fc1 =  tf.reshape(conv5,[-1,create_dense(conv5,1024)].get_shape().aslist()[0])
    fc1 = tf.add(tf.matmul(fc1,create_dense(conv5,1024)),create_biases(fc1))
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout((fc1, dropout))
    
    out = tf.add(tf.matmul(fc1,fc_to_out_weights(fc1,3)),create_biases(fc_to_out_weights(fc1,3)))
    return out

In [167]:
x = tf.placeholder(tf.float32,[None,512,512,3])
y = tf.placeholder(tf.float32,[None,n_classes])
keep_prob = tf.placeholder(tf.float32)

logits = cov_Net(x,keep_prob)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits,labels = y))

optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).mininize(cost)

correct_pred = tf.equal(tf.argmax(logits,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

init = tf.global_variables_initializer() 
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(epochs):
        for batch in range(train_tensors.shape[0]//batch_size):
            batch_x = train_tensors.next_batch1(train_tensors,batch,batch_size)
            batch_y = train_targets.next_batch1(train_targets,batch,batch_size)
            sess.run(optimizer,feed_dict={
                x:batch_x,
                y:batch_y,
                keep_prob:dropout
            })
            loss = sess.run(cost,feed_dict={
                x:batch_x,
                y:batch_y,
                keep_prob:1.
            })
            valid_acc = sess.run(accuracy,feed_dict={
                x:valid_tensors[:],
                y:valid_targets[:],
                keep_prob:1.
            })
            print('Epoch {:>2}, batch {:>3} -' 
                 'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                 epoch + 1,
                 batch + 1,
                 loss,
                 valid_acc))
        

ValueError: Shape must be rank 1 but is rank 0 for 'BiasAdd_5' (op: 'BiasAdd') with input shapes: [?,512,512,16], [].

In [ ]:
test_acc = sess.run(accuracy,feed_dict={
            x:test_tensors[:],
            y:test_targets[:],
            keep_prob:1.
        })
print('Testing Accuracy:{}'.format(test_acc))